In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import easyocr
import util

ModuleNotFoundError: No module named 'easyocr'

In [1]:
from ultralytics import YOLO

In [2]:
# Define constants for model and data paths
# These paths point to the configuration file, weights file, class names file, and input directory.
# Path to YOLO model configuration file

model_cfg_path = os.path.join('.', 'model', 'cfg', 'darknet-yolov3.cfg')

# Path to YOLO model weights file
model_weights_path = os.path.join('.', 'model', 'weights', 'model.weights')

# Path to file containing class names used by the model
class_names_path = os.path.join('.', 'model', 'class.names')

# Directory containing input images for license plate detection
input_dir = r'D:\PycharmProjects\license_plate_detection\data'
# Ensure to replace this path with the actual path to your dataset or input images.

# These paths are used to load the YOLO model and necessary files, as well as specify the input directory.
# Ensure that the directory structure and file names match the actual structure and names in your project.

In [3]:
net = cv2.dnn.readNetFromDarknet(model_cfg_path, model_weights_path)

In [1]:
reader = easyocr.Reader(['en'])
# Iterate through each image in the input directory
for img_name in os.listdir(input_dir):

    # Construct full path to the image
    img_path = os.path.join(input_dir, img_name)

    # Load class names from the file
    with open(class_names_path, 'r') as f:
        class_names = [j[:-1] for j in f.readlines() if len(j) > 2]
        f.close()

    # Load the image
    img = cv2.imread(img_path)

    # Check if the image is successfully loaded
    if img is None:
        print(f"Error: Unable to read image")
    else:
        # Get image dimensions
        H, W, _ = img.shape
        # Continue with the rest of the code

    # Convert the image to a format suitable for YOLO model
    blob = cv2.dnn.blobFromImage(img, 1 / 255, (416, 416), (0, 0, 0), True)

    # Get YOLO model detections
    net.setInput(blob)
    detections = util.get_outputs(net)

    # Lists to store bounding boxes, class IDs, and scores
    bboxes = []
    class_ids = []
    scores = []

    # Process each detection
    for detection in detections:
        # Extract coordinates of the bounding box
        bbox = detection[:4]
        xc, yc, w, h = bbox
        bbox = [int(xc * W), int(yc * H), int(w * W), int(h * H)]

        # Extract confidence score and class ID
        bbox_confidence = detection[4]
        class_id = np.argmax(detection[5:])
        score = np.amax(detection[5:])

        # Append to lists
        bboxes.append(bbox)
        class_ids.append(class_id)
        scores.append(score)

    # Apply Non-Maximum Suppression (NMS) to filter overlapping bounding boxes
    bboxes, class_ids, scores = util.NMS(bboxes, class_ids, scores)


    # Process each detected bounding box
    for bbox_, bbox in enumerate(bboxes):
        xc, yc, w, h = bbox

        # Extract the license plate region from the image
        license_plate = img[int(yc - (h / 2)):int(yc + (h / 2)), int(xc - (w / 2)):int(xc + (w / 2)), :].copy()

        # Draw a rectangle around the detected license plate on the original image
        img = cv2.rectangle(img,
                            (int(xc - (w / 2)), int(yc - (h / 2))),
                            (int(xc + (w / 2)), int(yc + (h / 2))),
                            (0, 255, 0), 15)

        # Convert the license plate region to grayscale
        license_plate_gray = cv2.cvtColor(license_plate, cv2.COLOR_BGR2GRAY)

        # Threshold the license plate image
        _, license_plate_thresh = cv2.threshold(license_plate_gray, 64, 255, cv2.THRESH_BINARY_INV)

        # Perform OCR on the thresholded license plate image
        output = reader.readtext(license_plate_thresh)

        license_plate_texts = []
        Avg_score = []

        # Process each OCR output
        for out in output:
            text_bbox, text, text_score = out
            if text_score > 0.2:
                license_plate_texts.append(text)
                Avg_score.append(text_score)
        print(img_path,(' ').join(license_plate_texts), np.mean(Avg_score))

NameError: name 'easyocr' is not defined

In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import easyocr
import torch
from torchvision import transforms
import ultralytics

# Define constants for model and data paths
model_path = r'D:\PycharmProjects\license_plate_detection\model_- 12 january 2024 14_47.pt'
input_dir = r'D:\PycharmProjects\license_plate_detection\data'

# List to store detected license plate texts
license_plate_texts = []



# Load your new license plate detection model
new_model = torch.load(model_path)
new_model.eval()

# Initialize EasyOCR reader for English language
reader = easyocr.Reader(['en'])

ModuleNotFoundError: No module named 'ultralytics'

In [7]:
import requests

# API URL, use actual MODEL_ID
url = f"https://api.ultralytics.com/v1/predict/kzp4sRWJYJKkgvPkeX70"

# Headers, use actual API_KEY
headers = {"x-api-key": "2386d451088da58c8638f6d14020fc3d1bbab835c0"}

# Inference arguments (optional)
data = {"size": 640, "confidence": 0.25, "iou": 0.45}

# Load image and send request
with open("D:\PycharmProjects\license_plate_detection\data1\Cars348.png", "rb") as image_file:
    files = {"image": image_file}
    response = requests.post(url, headers=headers, files=files, data=data)

print(response.json())

{'data': [], 'message': 'Inference complete.', 'success': True}


In [8]:
from ultralytics import YOLO

# Load model
model = YOLO('yolov8s.pt')

# Run inference
results = model('image.jpg')

# Print image.jpg results in JSON format
print(results[0].tojson())

ModuleNotFoundError: No module named 'ultralytics'